In [1]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [2]:
# Descargar el lexicón de VADER
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
# Inicializar el analizador de sentimiento de VADER
sia = SentimentIntensityAnalyzer()

In [4]:
# Ruta al archivo CSV
ruta_csv = r"C:\Users\rafae\MisCosas\IMDB\IMDB-Kaggle\IMDB Dataset.csv"

# Cargar el archivo CSV en un DataFrame de pandas
# Utilizamos 'chunk_size' si el archivo es muy grande para manejar en memoria
try:
    df = pd.read_csv(ruta_csv)
except FileNotFoundError:
    print(f"El archivo no se encontró en la ruta especificada: {ruta_csv}")
    exit()

In [5]:
# Función para determinar el sentimiento basado en el puntaje compuesto de VADER
def obtener_sentimiento(texto):
    puntajes = sia.polarity_scores(texto)
    compuesto = puntajes['compound']
    if compuesto >= 0.05:
        return 'positive'
    elif compuesto <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [6]:
# Aplicar la función de sentimiento a cada reseña
# Utilizamos tqdm para mostrar una barra de progreso
tqdm.pandas()

print("Analizando sentimientos...")
df['sentiment_vader'] = df['review'].progress_apply(obtener_sentimiento)

Analizando sentimientos...


100%|██████████| 50000/50000 [01:05<00:00, 768.59it/s]


In [7]:
# Opcional: Comparar con la etiqueta original
# Calcular precisión básica
# Suponiendo que solo hay 'positive' y 'negative' en la etiqueta original. Vader también puede dar una clasificación 'neutral' 
# así que no se considera en la comparación.
df['sentiment_vader_binary'] = df['sentiment_vader'].apply(
    lambda x: 'positive' if x == 'positive' else 'negative'
)

In [8]:
# Guardar los resultados en un nuevo archivo CSV
ruta_salida = r"C:\Users\rafae\MisCosas\IMDB\IMDB-Kaggle\IMDB_Dataset_Con_Sentimiento_VADER.csv"
df.to_csv(ruta_salida, index=False)
print(f"Resultados guardados en: {ruta_salida}")

Resultados guardados en: C:\Users\rafae\MisCosas\IMDB\IMDB-Kaggle\IMDB_Dataset_Con_Sentimiento_VADER.csv


In [10]:
# Convertir etiquetas a valores binarios si no lo están
# Por ejemplo, 'positive' -> 1 y 'negative' -> 0
df['sentiment_binary'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df['sentiment_vader_binary'] = df['sentiment_vader_binary'].map({'positive': 1, 'negative': 0})

In [12]:
# Definir variables de etiquetas reales y predichas
y_true = df['sentiment_binary']
y_pred = df['sentiment_vader_binary']

In [13]:
# Calcular precisión
precision = precision_score(y_true, y_pred)

# Calcular recall
recall = recall_score(y_true, y_pred)

# Calcular F1-score
f1 = f1_score(y_true, y_pred)

# Calcular exactitud
exactitud = accuracy_score(y_true, y_pred)

# Calcular matriz de confusión
matriz_confusion = confusion_matrix(y_true, y_pred)

In [14]:
# Mostrar los resultados
print(f"Precisión: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Exactitud: {exactitud:.4f}")
print("Matriz de Confusión:")
print(matriz_confusion)


Precisión: 0.6505
Recall: 0.8535
F1-score: 0.7383
Exactitud: 0.6974
Matriz de Confusión:
[[13534 11466]
 [ 3663 21337]]


In [15]:
# Opcional: Mostrar reporte de clasificación completo
reporte = classification_report(y_true, y_pred, target_names=['Negative', 'Positive'])
print("\nReporte de Clasificación:")
print(reporte)


Reporte de Clasificación:
              precision    recall  f1-score   support

    Negative       0.79      0.54      0.64     25000
    Positive       0.65      0.85      0.74     25000

    accuracy                           0.70     50000
   macro avg       0.72      0.70      0.69     50000
weighted avg       0.72      0.70      0.69     50000

